In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 25 14:31:01 2019
@author: minyoungpark

Moved to Jupyter notebook (for better or worse) on 12/03/2020 by Joseph
Updated code as well to streamline 3D reconstruction
"""
import sys

# add folder with calibration functions to path
calib_folder = r'D:\Lab\GIT\proc-joe\Limblab_DLC\cam_calib_20200508'

# set project folder 
project_folder = r'D:\Lab\Data\DLC_videos\Han_20201204_rwFreeReach\Han_reaching-Joe-2020-12-10'

# determine if we are using filtered data or not
use_filtered_data = True
remove_triangulation = False

# using reference frame or use an arbitrary frame?
use_reference_frame = True

# imports and other stuff
sys.path.append(calib_folder)

from utils.utils import load_config
from calibration.intrinsic import calibrate_intrinsic
from calibration.extrinsic import calibrate_extrinsic
from triangulation.triangulate import reconstruct_3d
from utils.vis_utils import generate_three_dim_video
from utils.vis_utils import generate_three_dim_pictures
from utils.triangulation_utils import add_static_points
    
import numpy as np
import toml
import glob
import os
import yaml

In [4]:
# setup config file 
# load basic toml folder and then fill out relevant entries
parsed_toml = toml.load(calib_folder + r'\config_master.toml')

# upate calib video path and prefix and extension
parsed_toml['calibration']['calib_video_path'] = project_folder + r'\videos\calib'
parsed_toml['calibration']['calib_video_prefix'] = 'Han_20201204_0000'

# set number of camers
n_cams = 4


# update paths to 2d data while removing (or keeping) videos with filtered
if(use_filtered_data):
    vid_list = glob.glob(project_folder + r'\videos\*filtered.csv')
else:
    vid_list_temp = glob.glob(project_folder + r'\videos\*.csv')
    vid_list = []
    for vid_name in vid_list_temp:
        if vid_name.find('filtered') == -1:
            vid_list.append(vid_name)
    
parsed_toml['paths_to_2d_data'] = vid_list
    
# update path to save static data
parsed_toml['path_to_save_static_data'] = project_folder + r'\videos'

# update output video path
parsed_toml['output_video_path'] = project_folder + r'\reconstructed-3d-data'

# update triangulation data (or remove if desired)
if(remove_triangulation):
    parsed_toml['triangulation'].pop('axes', None)
    parsed_toml['triangulation'].pop('reference_point', None)
# else, likely leave alone. Could fill this in if we change how we do reference points

# update reconstruction output path and threshold
parsed_toml['triangulation']['reconstruction_threshold'] = 0.7 # 0.7 default
parsed_toml['triangulation']['reconstruction_output_path'] = project_folder + r'\reconstructed-3d-data'

# update labeling scheme and bodyparts interested
# only need to update if using something other than base arm points
parsed_toml['labeling']['scheme'] = []
parsed_toml['labeling']['bodyparts_interested'] = ['shoulder','elbow1','elbow2','wrist1','wrist2','hand1','hand2','hand3','pointX','pointY','pointZ']

toml_config_file = project_folder + r'\recon_config.toml'

with open(toml_config_file,'w+') as file:
    toml.dump(parsed_toml,file)


In [16]:
config = load_config(toml_config_file)

#%% If you already ran calibration you don't need to run these.
calibrate_intrinsic(config)
# calibrate intrinsic runs for each camera, so we can run it on the full video list



# calibrate extrinsic
calibrate_extrinsic(config)


intrinsics_1.toml already exists.

intrinsics_2.toml already exists.

intrinsics_3.toml already exists.

intrinsics_4.toml already exists.

extrinsics.toml already exists.


In [5]:
if(use_reference_frame):
    labels = ['pointX', 'pointY', 'pointZ']
    
    snapshots = vid_list

    # initialize static
    static = {label : [] for label in labels}
        
    # get labeled reference point for each camera and store in a new file, also copy over other tracking data
    # make sure to overwrite pointX, pointY, and pointZ if already exist
    # labeled reference point is in each csv file in each folder in project_folder + 'labeled-data'
    labeled_folder = project_folder + r'\labeled-data'
    
    video_files = config['paths_to_2d_data']
    pointX_data = []; pointY_data = []; pointZ_data = [];
    
    for vid_idx in range(len(video_files)):
        video_name = os.path.split(video_files[vid_idx])[-1]
        DLC_matches = re.finditer("DLC", video_name)
        DLC_idx = [match.start() for match in DLC_matches]
        DLC_idx = DLC_idx[-1] # use last one
    
        video_prefix = video_name[:DLC_idx]
    
        # only for today's testing session because we are doing 3D reconstruction on different data than we trained the model
        video_prefix = video_prefix + "_trimmed"
        
        path_to_folder = labeled_folder + '\\' + video_prefix
        csv_file = glob.glob(path_to_folder+ '\\*.csv')[0]
        data = pd.read_csv(csv_file, header=[1,2], index_col=0)
    
        # get (x,y) for each label Store in Static properly
        
        for label in labels:
            x = data[(label, 'x')].values[~np.isnan(data[(label, 'x')].values)]
            y = data[(label, 'y')].values[~np.isnan(data[(label, 'y')].values)]
            x = x[0] # only get first labels
            y = y[0]
            static[label].append([x,y])

    
    add_static_points(config, labels, static, snapshots)

    
    

In [18]:
if(not os.path.isdir(parsed_toml['triangulation']['reconstruction_output_path'])):
    os.mkdir(parsed_toml['triangulation']['reconstruction_output_path'])
    
recovery = reconstruct_3d(config)

100%|██████████████████████████| 29492/29492 [00:33<00:00, 881.70it/s]


In [17]:
config

{'paths_to_2d_data': ['D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\videos\\Han_20201204_00005DLC_resnet50_Han_202012Dec4shuffle1_1030000_filtered.csv',
  'D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\videos\\Han_20201204_00006DLC_resnet50_Han_202012Dec4shuffle1_1030000_filtered.csv',
  'D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\videos\\Han_20201204_00007DLC_resnet50_Han_202012Dec4shuffle1_1030000_filtered.csv',
  'D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\videos\\Han_20201204_00008DLC_resnet50_Han_202012Dec4shuffle1_1030000_filtered.csv'],
 'path_to_save_static_data': 'D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\videos',
 'output_video_path': 'D:\\Lab\\Data\\DLC_videos\\Han_20201204_rwFreeReach\\Han_reaching-Joe-2020-12-10\\reconstructed-3d-data',
 'video': {'fps': 25, 'resolution': [1280, 1024]}

In [ ]:
# =============================================================================
# #%% Save 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# class NumpyArrayEncoder(JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return JSONEncoder.default(self, obj)
# with open("pop_0610_anipose.json", "w") as write_file:
#     json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#     
# #%% Load 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# with open("pop_0317_3.json", "r") as read_file:
#     print("Converting JSON encoded data into Numpy array")
#     recovery = json.load(read_file)
# recovery['registration_mat'] = np.array(recovery['registration_mat'])
# recovery['center'] = np.array(recovery['center'])
# =============================================================================


#%% Save 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
with open("pop_0610_anipose.json", "w") as write_file:
    json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#%% Load 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
with open("pop_0317_3.json", "r") as read_file:
    print("Converting JSON encoded data into Numpy array")
    recovery = json.load(read_file)
recovery['registration_mat'] = np.array(recovery['registration_mat'])
recovery['center'] = np.array(recovery['center'])

#%% generate 3D picture to find the optimal azimuth and elevation value
generate_three_dim_pictures(config)

#%% Try 3D stick figure video

generate_three_dim_video(config)

#%% Testing if the generated 3D results makes sense, by calculating the distance between wrist and hand
"""
import pandas as pd
import numpy as np
from numpy import array as arr

data_path = 'C:/Users/dongq/DeepLabCut/Han-Qiwei-2020-02-21/3D-data/output_3d_data_rotate4.csv'

df = pd.read_csv(data_path)
wrist = np.empty((len(df), 3))
hand = np.empty((len(df), 3))

wrist[:,0] = arr(df['wrist1_x'])
wrist[:,1] = arr(df['wrist1_y'])
wrist[:,2] = arr(df['wrist1_z'])


hand[:,0] = arr(df['hand2_x'])
hand[:,1] = arr(df['hand2_y'])
hand[:,2] = arr(df['hand2_z'])

dist = wrist - hand
dist_finite = dist[np.isfinite(dist[:,0]), :]
dist_3d = np.linalg.norm(dist_finite, axis=1)
print(np.median(dist_3d))
"""